In [ ]:
import anthropic
import openai
import ipywidgets as widgets
from IPython.display import display, Markdown, update_display
from dotenv import load_dotenv
import requests
import json

MODEL_CLAUDE = 'claude-3-5-sonnet-20241022'
MODEL_LLAMA = 'llama3.2'
MODEL_GPT = 'gpt-4o-mini'

load_dotenv()

# Define models
models = [
    ('Claude (Anthropic)', MODEL_CLAUDE),
    ('LLaMA (Meta)', MODEL_LLAMA),
    ('GPT (OpenAI)', MODEL_GPT)
]

model_dropdown = widgets.Dropdown(
    options=[('', None)] + [(model[0], model[0]) for model in models],
    value=None,
    placeholder='Choose a model',
    description='Model:',
    style={'description_width': 'initial'}
)

selected_model = ""

text = input(f"Hello, I am your personal tutor. Please ask me a question regarding your code:")

system_prompt = "You are a helpful technical tutor who answers questions about programming, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + text

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Get gpt-4o-mini to answer, with streaming
def get_gpt_response():
    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(f"**Question:** {text}\n\n**Answer:** {response}"), display_id=display_handle.display_id)
    return response

# Get Llama 3.2 to answer, with streaming
def get_llama_response():
    api_url = "http://localhost:11434/api/chat"
    payload = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": True
    }
    response = requests.post(api_url, json=payload, stream=True)
    display_handle = display(Markdown(""), display_id=True)
    result = ""
    
    for line in response.iter_lines():
        if line:
            json_response = json.loads(line)
            if "message" in json_response:
                content = json_response["message"].get("content", "")
                result += content
                update_display(Markdown(f"**Question:** {text}\n\n**Answer:** {result}"), display_id=display_handle.display_id)
            if json_response.get("done", False):
                break
    
    return result

# Get Claude 3.5 to answer, with streaming
def get_claude_response():
    client = anthropic.Anthropic()

    response = client.messages.create(
        model=MODEL_CLAUDE,
        system=system_prompt,
        messages=[
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        stream=True,
        max_tokens=8192,
        temperature=1,
    )
    result = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in response:
        # Check if the chunk is a ContentBlockDeltaEvent
        if hasattr(chunk, 'delta') and hasattr(chunk.delta, 'text'):
            result += chunk.delta.text
            update_display(Markdown(f"**Question:** {text}\n\n**Answer:** {result}"),  display_id=display_handle.display_id)
    return result

def on_text_submit():
    try:
        if 'Claude' in selected_model:
            display(Markdown(f"# **Selected model: {selected_model}**"))
            get_claude_response()
        elif 'LLaMA' in selected_model:
            display(Markdown(f"# **Selected model: {selected_model}**"))
            get_llama_response()
        elif 'GPT' in selected_model:
            display(Markdown(f"# **Selected model: {selected_model}**"))
            get_gpt_response()
    except Exception as e:
        display(Markdown(f"**Error:** {str(e)}"))

def on_model_select(change):
    global selected_model

    selected_model = change['new'].split(' ')[0]
    if selected_model is not None:
        on_text_submit()
    return change['new'].split(' ')[0]

# Register callbacks
model_dropdown.observe(on_model_select, names='value')

display(model_dropdown)